# __<font color='red'> Stage : Etude et analyse critique d'un modèle de réponse dynamique de muscle  </font>__

---

**But : Décrire le modèle du muscle de façon thermodynamique et montrer que celui-ci reprends le modèle de Hill.**

---

Le muscle peut être représenté par une machine thermodynamique. En effet cette machine thermodynamique est ouverte, et donc échange de la matière et de l'énergie avec l'extérieur. Ces élements sont échangés avec une conductance, et donc peuvent être représentés par leurs inverse, leurs résistances. Cette machine est assez spéciale et fonctionne avec des réservoirs de potentiel chimique, là où, classiquement, nous aurions des réservoirs d'énergie. 

Il existe donc un flux d'énergie qui arrive depuis le réservoir de potentiel chimique $µ_+$, passe par une résistance $R_+$ traversant une zone de conversion dite **zone métabolique de conversion**, depuis laquelle est délivrée une puissance $P$, donc une intensité, appelée intensité métabolique $i_m$, ce flux deviens un flux d'énergie $\phi_-$, passe par une résistance $R_-$ puis finit sa course dans le puit, de potentiel chimique $µ_-$. (Voir figure ci-dessous, reprise de l'article)

![muscle](figure_meta.png)

**Explications du modèle de Hill**

En 1938, Archibald Vivian Hill, propose dans son papier un modèle de contraction de muscle, reliant la force de la charge appliquée sur le muscle à la vitesse de contraction de celui-ci. Il apparait une force maximale isométrique générée par le muscle, ainsi que 2 constantes $a$ et $b$.  $$ (F+a)(v+b) = b(F_0 + a)$$
+ $F$ est la charge sur le muscle
+ $F_0$ est la charge isométrique
+ $a$ : coefficient de raccourcissement de la chaleur (coefficient of shortening heat)
+ $b : a \frac{v_0}{F_0}$

**Modélisation numérique**

In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, interactive
import ipywidgets
import matplotlib.pyplot as plt

In [2]:
# definitions des fonctions

 #Force en fonction de la vitesse
def Fo(v,a,b,F0):
    return (b*(F0+a)/(v+b)) - a

 #Vecteurs
def fonc(a,b,F0):
    v = np.arange(0,1,0.01)
    F = Fo(v,a, b, F0)

    #Tracé
    fig = go.Figure(
        data=[go.Bar(x=v, y=F)],
        layout=dict(title=dict(text="Force en fonction de la vitesse")),
    )
    fig.update_xaxes(title="v en m/s")
    fig.update_yaxes(title="Force (unités arbitraires)")
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
    fig.show()

In [3]:

                  #Tracé dynamique : 

interact = interactive(fonc, a = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=10.0,
    step=0.1,
    description='a:',
), b = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=10.0,
    step=0.1,
    description='b:',
), F0 = widgets.FloatSlider(value=1.0,
    min=0,
    max=10.0,
    step=0.1,
    description='$F_0$:'
) ) 


interact

interactive(children=(FloatSlider(value=1.0, description='a:', max=10.0), FloatSlider(value=1.0, description='…

**Retour sur le modèle thermodynamique de la conversion d'énergie dans un muscle**

La conversion énergie chimique - énergie mécanique se fait selon certains paramètres. Pour plus de facilité nous prenons le cas d'efforts à durée limitée. Nous allons essayer de tracer les flux d'énergie ainsi que la puissance délivrée en fonction de l'intensité métabolique. De part ce fait nous pourrons montrer que ce nouveau modèle reprends le modèle de Hill.

Les équations pour les flux $\phi_+$ et $\phi_-$ sont donnée par : 
+ $\phi_+ = \frac{\frac{F_{iso} I_m}{\eta_c}+B}{I_t + I_m}I_t$ 
+ $\phi_- = R_m I_m^{2} + \frac{R_{bf}  I_m^{2} + F_{iso}(\frac{1}{\eta_c}-1)I_m + B}{I_t + I_m}$

La puissance est donnée par :

+ $P_m = [F_{iso} - (R_m + R_H(I_m))I_m]I_m$

Avec :
+ $R_m$ la résistance métabolique dissipative
+ $R_{H}$ la résistance métabolique dépendante de l'intensité : $R_{H} = \frac{F_{iso} + R_{bf} I_{T}}{I_{T} + I_{M}} $

+ $F_{iso}$ force isométrique
+ $I_m$ intensité métabolique
+ $\eta_{c}$ rendement
+ $I_{t}$ intensité seuil
+ $R_{bf}$ resistance feedback
+ $B$ Basal




Représentons ces flux d'énergie ainsi que la puissance

In [4]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, interactive
import ipywidgets
import matplotlib.pyplot as plt

In [5]:
#Fonctions: 
def f(I_m,F,R_fb,I_t,R_m,B,etaC) :
    Rh=(F+R_fb*I_t)/(I_t+I_m) #Resistance métabolique dépendante de l'intensité métabolique
    fp=(F*I_m/etaC+B)*I_t/(I_t+I_m) # phi+
    fm=R_m*I_m*I_m+(R_fb*I_m*I_m+F*(1/etaC-1)*I_m+B)*I_t/(I_t+I_m) # phi-
    p=(F-(R_m+Rh)*I_m)*I_m #Puissance délivrée dirant un effort physique
    cot=fm/(I_m+1e-10) #Cost of transport
    eta = (fp - fm)/fp
    return [fp,fm,p,cot,eta]

def tracepmp(F,R_fb,I_t,R_m,B,etaC): #fonction qui trace phi+,phi-, puissance
    I_m = np.arange(0, 14, 0.1)
    plt.figure(figsize= (6,5))
    plt.plot(I_m, f(I_m,F,R_fb,I_t,R_m,B,etaC)[0],color = 'b', label = r'$\phi_+$')  #plot phi+
    plt.plot(I_m, f(I_m,F,R_fb,I_t,R_m,B,etaC)[1], color = 'r',label = r'$\phi_-$')  #plot phi-
    plt.plot(I_m, f(I_m,F,R_fb,I_t,R_m,B,etaC)[2], color = 'k',label = 'Power')  #plot puissance
    plt.xlabel('Intensité métabolique')
    plt.ylabel('Puissance [Unités Arbitraires]')
    plt.legend()
    plt.show()
    
def trace_rendement(F,R_fb,I_t,R_m,B,etaC):
    I_m = np.arange(0, 14, 0.1)
    fp = f(I_m,F,R_fb,I_t,R_m,B,etaC)[0]
    fm = f(I_m,F,R_fb,I_t,R_m,B,etaC)[1]
    eta = f(I_m,F,R_fb,I_t,R_m,B,etaC)[-1]
    P = (fp-fm)
    
    max_eta = eta.max(), np.where(eta == eta.max())[0][0]
    max_P = P.max(), np.where(P == P.max())[0][0]
    
    plt.figure(figsize = (6,6))
    
    plt.subplot(211)
    plt.plot(I_m, eta, color = "r")
    plt.xlabel(r'$I_m$')
    plt.ylabel("$\eta$")
    
    
    plt.subplot(212)
    plt.plot(P,eta, color = "b")
    plt.scatter(P[max_eta[1]],max_eta[0],label='Rendement maximal', color='k')
    plt.scatter(max_P[0], eta[max_P[1]],label ='Puissance maximale', color='r')
    plt.xlabel("Puissance [unités arbitraires]")
    plt.ylabel("$\eta$")
    plt.legend()
    plt.show()

In [6]:
interact = interactive(tracepmp,
    F = widgets.FloatSlider(                     #Force isométrique
    value=14.70,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$F_{iso}$"
),
    R_fb= widgets.FloatSlider(                   #Resistance de FeedBack
    value=0.10,
    min=0.1,
    max=10.0,
    step=0.1,
    description="$R_{fb}$"
), 
    I_t= widgets.FloatSlider(                    # Itensité métabolique seuil (Treshold intensity)
    value=10.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$I_T$"
),
    
    R_m= widgets.FloatSlider(                    # Résistance métabolique dissipative
    value=0.40,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$R_M$"
),  
    B= widgets.FloatSlider(                      # Basal
    value=3.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$B$"
),                   
    etaC= widgets.FloatSlider(                   # Rendement
    value=0.84,
    min=0.0001,
    max=1,
    step=0.0001,
    description="$\eta_C$"
) ) 


interact

interactive(children=(FloatSlider(value=14.7, description='$F_{iso}$', min=0.1), FloatSlider(value=0.1, descri…

In [7]:
interact2 = interactive(trace_rendement,
    F = widgets.FloatSlider(                        #Force isométrique
    value=14.70,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$F_{iso}$"
),
    R_fb= widgets.FloatSlider(                      #Resistance de FeedBack
    value=0.10,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$R_{fb}$"
), 
    I_t= widgets.FloatSlider(                       # Itensité métabolique seuil (Treshold intensity)
    value=10.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$I_T$"
), 
    R_m= widgets.FloatSlider(                       # Résistance métabolique dissipative
    value=0.40,
    min=0.1,
    max=3.0,
    step=0.1,
    description="$R_M$"
),  
    B= widgets.FloatSlider(                         # Basal
    value=3.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$B$"
),                   
    etaC= widgets.FloatSlider(                      # Rendement
    value=0.84,
    min=0.0001,
    max=1,
    step=0.0001,
    description="$\eta_C$"
) ) 


interact2

interactive(children=(FloatSlider(value=14.7, description='$F_{iso}$', min=0.1), FloatSlider(value=0.1, descri…

**Tracé de la force à partir de la puissance**\
Avec la courbe de puissance, nous pouvons en tirer la force métabolique par la relation $P_{m} = F_{M}I_{M}$.\
On peux donc tracer la force en fonction de l'intensité métabolique.

In [8]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, interactive
import ipywidgets
import matplotlib.pyplot as plt

In [9]:
#Tracé de la force à partir de la puissance:
def forcefrompower(I_m,F,R_fb,I_t,R_m,B,etaC) :
    Rh=(F+R_fb*I_t)/(I_t+I_m) #Resistance métabolique dépendante de l'intensité métabolique
    p=(F-(R_m+Rh)*I_m)*I_m #Puissance délivrée dirant un effort physique
    FFP = p/(I_m+1e-10) #FFP : Force From Power, puissance divisée par l'intensité métabolique
    return FFP

def traceFFP(F,R_fb,I_t,R_m,B,etaC):
    I_m = np.arange(0, 14, 0.1)
    FFP = forcefrompower(I_m,F,R_fb,I_t,R_m,B,etaC)
    fig = go.Figure(
        data=[go.Bar(x=I_m, y=FFP)],
        layout=dict(title=dict(text="Force en fonction de l'intensité métabolique")),
    )
    fig.update_xaxes(title=r"$I_{m} \propto v$")
    fig.update_yaxes(title="Force (unités arbitraires)")
    fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
    fig.show()

In [10]:
interactforce = interactive(traceFFP,
    F = widgets.FloatSlider(                        #Force isométrique
    value=45.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$F_{iso}$"
),
    R_fb= widgets.FloatSlider(                      #Resistance de FeedBack
    value=0.10,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$R_{fb}$"
), 
    I_t= widgets.FloatSlider(                       # Itensité métabolique seuil (Treshold intensity)
    value=10.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$I_T$"
), 
    R_m= widgets.FloatSlider(                       # Résistance métabolique dissipative
    value=1.30,
    min=0.1,
    max=3.0,
    step=0.1,
    description="$R_M$"
),  
    B= widgets.FloatSlider(                         # Basal
    value=16.0,
    min=0.1,
    max=100.0,
    step=0.1,
    description="$B$"
),                   
    etaC= widgets.FloatSlider(                      # Rendement
    value=0.84,
    min=0.0001,
    max=1,
    step=0.0001,
    description="$\eta_C$"
) ) 


interactforce

interactive(children=(FloatSlider(value=45.0, description='$F_{iso}$', min=0.1), FloatSlider(value=0.1, descri…